# Processor System Design Program #

In [ ]:
from pynq import Overlay, MMIO
import xrfdc
import xrfclk
import numpy as np
import time

base = Overlay('/home/xilinx/pynq/overlays/PSOverlay/PSOverlay.bit')

Initialize clocks

In [ ]:
xrfclk.set_ref_clks(lmk_freq = 500.00, lmx_freq = 500.00) # Initialize clocks
while base.usp_rf_data_converter_0.IPStatus['DACTileStatus'][0]['TileState']!=15:
    xrfclk.set_ref_clks(lmk_freq = 500.00, lmx_freq = 500.00)
    time.sleep(3)

Parameters and functions definition:

In [ ]:
# Parameters
Theta = [10,20,30,40]
Phi = [10,20,30,40]
num_angles = len(Theta)
Radius = 1
lambda_val = 1.0
PosX = [0, 0.5]
PosY = [0, 0]
num_pos =len(PosX)
Steering = [0,0]

# Antenna Steering Array function
def get_steering(ThetaO, PhiO, n):
    for i in range(n):
        if np.sqrt(PosX[i]**2 + PosY[i]**2) <= Radius:
            Steering[i] = -2*np.pi*np.sin(np.radians(ThetaO))*(PosX[i]*np.cos(np.radians(PhiO))+PosY[i]*np.sin(np.radians(PhiO)))
    return 0

In [ ]:
# Map PL GPIO
new_params_button = base.hier_GPIO.axi_gpio_2[0]
dacA_enable = base.dac_generator.axi_gpio_4[0]
dacB_enable = base.dac_generator.axi_gpio_3[0]
RAMB_limit = base.dac_generator.axi_gpio_0
start = base.hier_GPIO.axi_gpio_3[0] #PMOD0_0
end = base.hier_GPIO.axi_gpio_3[1] #PMOD0_1

# DAC player memory
DAC_B_BASEADDR = base.mem_dict["dac_generator/axi_bram_ctrl_0"]["phys_addr"]
DAC_B_ADDRRANGE = base.mem_dict["dac_generator/axi_bram_ctrl_0"]["addr_range"]
DAC_B_MMIO = MMIO(DAC_B_BASEADDR,DAC_B_ADDRRANGE)
DAC_A_BASEADDR = base.mem_dict["dac_generator/axi_bram_ctrl_1"]["phys_addr"]
DAC_A_ADDRRANGE = base.mem_dict["dac_generator/axi_bram_ctrl_1"]["addr_range"]
DAC_A_MMIO = MMIO(DAC_A_BASEADDR,DAC_A_ADDRRANGE)

dacA_player = DAC_A_MMIO.array[0:DAC_A_MMIO.length].view("int16") 
dacB_player = DAC_B_MMIO.array[0:DAC_B_MMIO.length].view("int16") 

In [ ]:
DAC_SR = 6.5E9  # Sample rate of DACs 6.5 GHz
Fc = 4.875E9 # Set center frequency of waveform to 5 GHz
DAC_Amplitude = 16383.0  # 14bit DAC +16383/-16384
X_axis = (1/DAC_SR) * np.arange(0,dacB_player.shape[0])
ramlimitvalue=0xFFF

# Set address limit for RAM B
RAMB_limit.write(0,ramlimitvalue)

DAC_sinewave = DAC_Amplitude * np.cos(2*np.pi*Fc*X_axis)
dacA_player[:] = np.int16(DAC_sinewave)
dacB_player[:] = np.int16(DAC_sinewave)
dacB_enable.off()
dacA_enable.off()
dacB_enable.on()
dacA_enable.on()

Main Loop

In [ ]:
new_params_button_prev = 0
angles_count = 0
result = 0

while(1):
    if new_params_button.read()==1 and new_params_button_prev == 0:
        new_params_button_prev = 1
        start.write(1)
        if angles_count < num_angles-1:
            angles_count+=1
        else: angles_count = 0
        result = get_steering(Theta[angles_count], Phi[angles_count], num_pos)
        start.write(result)
        print(Steering)
    elif new_params_button.read() == 0: 
        new_params_button_prev = 0
        end.write(0)
        result = 0
     